# Sprint 2 - Core Primal Numerical Engine

This notebook demonstrates:
- state-space meshing
- FDM operator construction
- PSOR obstacle solve in primal space
- convergence against analytical baseline

**Roadmap:** Sprint 2 (Weeks 3-4).

In [ ]:
import numpy as np

from optistop import GBM, Grid, PSORSolver, PrimalObstacleSolver, VanillaCall, McDonaldSiegelAnalytical
from optistop.solvers.fdm import FDMOperator

In [ ]:
# Problem setup
r, mu, sigma, I = 0.05, 0.02, 0.20, 1.0

process = GBM(mu=mu, sigma=sigma)
utility = VanillaCall(investment_cost=I)
grid = Grid(x_min=1e-4, x_max=8.0, n=2500)
operator = FDMOperator(process=process, r=r)
psor = PSORSolver(omega=1.2, tol=1e-7, max_iter=50_000)

result = PrimalObstacleSolver(grid=grid, utility=utility, operator=operator, psor=psor).solve()
print('Numerical trigger x* =', result.trigger)
print('Converged =', result.converged, '| iterations =', result.iterations)
print('Method =', result.metadata.get('method'))

In [ ]:
# Compare with analytical benchmark
x_star_ref = McDonaldSiegelAnalytical(r=r, mu=mu, sigma=sigma, investment_cost=I).trigger()
rel_error = abs(result.trigger - x_star_ref) / x_star_ref

print(f'Analytical trigger x* = {x_star_ref:.8f}')
print(f'Relative error = {rel_error:.6%}')

In [ ]:
# Grid-convergence mini-study
for n in [500, 1000, 2000, 3000]:
    g = Grid(x_min=1e-4, x_max=8.0, n=n)
    res = PrimalObstacleSolver(grid=g, utility=utility, operator=operator, psor=psor).solve()
    e = abs(res.trigger - x_star_ref) / x_star_ref
    print(f'N={n:4d} | trigger={res.trigger:.8f} | rel_error={e:.6%}')

## Sprint 2 Deliverable Check

- [x] FDM sparse operator assembled
- [x] PSOR obstacle solver operational
- [x] Numerical convergence vs analytical baseline demonstrated